<a href="https://colab.research.google.com/github/RashmiSunkara/Restaurant-Chatbot/blob/main/PBL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Loading JSON File

In [ ]:
!pip install tflearn

     |████████████████████████████████| 107 kB 4.4 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=8c76e6418ee43c5bf435fc691086b0ea9bde140fe1deb30d9c7140a6ee768b11
  Stored in directory: /root/.cache/pip/wheels/5f/14/2e/1d8e28cc47a5a931a2fb82438c9e37ef9246cc6a3774520271
Successfully built tflearn


In [ ]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy
import tflearn
import tensorflow
import random

import json
with open('/content/drive/MyDrive/Colab Notebooks/intents.json') as file:
    data = json.load(file)

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
nltk.download('punkt')
words = []
labels = []
docs_x = []
docs_y = []
for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


WORD STEMMING

In [ ]:
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

BAG OF WORDS

In [ ]:
training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)
training = numpy.array(training)
output = numpy.array(output)

DEVELOPING THE MODEL

In [ ]:
tensorflow.compat.v1.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
model.fit(training, output, n_epoch=100, batch_size=8, show_metric=True)
model.save("model.tflearn")

Training Step: 399  | total loss: 1.59660 | time: 0.029s
| Adam | epoch: 100 | loss: 1.59660 - acc: 0.3633 -- iter: 24/28
Training Step: 400  | total loss: 1.55610 | time: 0.036s
| Adam | epoch: 100 | loss: 1.55610 - acc: 0.3770 -- iter: 28/28
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


MAKING PREDICTIONS

In [ ]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1

    return numpy.array(bag)


def chat():
    print("BOT: Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("CUSTOMER: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print(random.choice(responses))


chat()

BOT: Start talking with the bot (type quit to stop)!
CUSTOMER: Order Noodles
See you later, thanks for visiting
CUSTOMER: Menu
Bye! Come back again soon.
CUSTOMER: quit
